## Deliverable 2. Create a Customer Travel Destinations Map.

In [71]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,77.05,73,40,9.22,scattered clouds
1,1,Jamestown,US,42.0970,-79.2353,85.32,44,1,8.05,clear sky
2,2,Ochakiv,UA,46.6128,31.5498,60.94,64,48,7.43,scattered clouds
3,3,Chokurdakh,RU,70.6333,147.9167,59.70,60,48,3.49,scattered clouds
4,4,Kodinsk,RU,58.6881,99.1844,43.16,60,100,3.22,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)
print(preferred_cities_df.count())

City_ID                260
City                   260
Country                257
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64


In [11]:
# 4a. Determine if there are any empty rows.
empty_city_rows = preferred_cities_df.isna().sum().sum()
print("Empty rows: " + str(empty_city_rows))

Empty rows: 3


In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
print(clean_df.count())

City_ID                257
City                   257
Country                257
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
dtype: int64


In [15]:
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Ochakiv,UA,46.6128,31.5498,60.94,64,48,7.43,scattered clouds
3,3,Chokurdakh,RU,70.6333,147.9167,59.70,60,48,3.49,scattered clouds
6,6,Bengkulu,ID,-3.8004,102.2655,74.98,92,18,3.53,few clouds
7,7,Haines Junction,CA,60.7522,-137.5108,60.75,39,79,11.50,light rain
10,10,Saint-Philippe,RE,-21.3585,55.7679,73.47,82,19,2.01,light rain


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ochakiv,UA,60.94,scattered clouds,46.6128,31.5498,
3,Chokurdakh,RU,59.70,scattered clouds,70.6333,147.9167,
6,Bengkulu,ID,74.98,few clouds,-3.8004,102.2655,
7,Haines Junction,CA,60.75,light rain,60.7522,-137.5108,
10,Saint-Philippe,RE,73.47,light rain,-21.3585,55.7679,
11,Castro,BR,59.61,clear sky,-24.7911,-50.0119,
14,Souillac,MU,70.18,scattered clouds,-20.5167,57.5167,
19,Vila Franca Do Campo,PT,64.81,broken clouds,37.7167,-25.4333,
29,Mar Del Plata,AR,55.87,clear sky,-38.0023,-57.5575,
37,Barcelos,PT,68.29,clear sky,41.5388,-8.6151,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ochakiv,UA,60.94,scattered clouds,46.6128,31.5498,Briz
3,Chokurdakh,RU,59.70,scattered clouds,70.6333,147.9167,
6,Bengkulu,ID,74.98,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
7,Haines Junction,CA,60.75,light rain,60.7522,-137.5108,Parkside Inn
10,Saint-Philippe,RE,73.47,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Castro,BR,59.61,clear sky,-24.7911,-50.0119,CHACARA BAILLY
14,Souillac,MU,70.18,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
19,Vila Franca Do Campo,PT,64.81,broken clouds,37.7167,-25.4333,Pestana Bahia Praia
29,Mar Del Plata,AR,55.87,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
37,Barcelos,PT,68.29,clear sky,41.5388,-8.6151,Hotel Do Terço


In [60]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ochakiv,UA,60.94,scattered clouds,46.6128,31.5498,Briz
6,Bengkulu,ID,74.98,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
7,Haines Junction,CA,60.75,light rain,60.7522,-137.5108,Parkside Inn
10,Saint-Philippe,RE,73.47,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Castro,BR,59.61,clear sky,-24.7911,-50.0119,CHACARA BAILLY
14,Souillac,MU,70.18,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
19,Vila Franca Do Campo,PT,64.81,broken clouds,37.7167,-25.4333,Pestana Bahia Praia
29,Mar Del Plata,AR,55.87,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
37,Barcelos,PT,68.29,clear sky,41.5388,-8.6151,Hotel Do Terço
38,Hay River,CA,55.38,fog,60.8156,-115.7999,Ptarmigan Inn


In [59]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))